In [22]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import PCA
# import gower

In [2]:
bb_usage = pd.read_csv("data/user_profile/BB_Usage_CF.csv")

In [3]:
user_all_df = pd.read_csv("data/user_profile/account_no_map_all.csv")

In [4]:
user_profile = pd.read_csv("data/user_profile/prepared/User_Profile_Null_Handled.csv")

In [5]:
user_bb_map = pd.read_csv("data/user_profile/User_BB_Package_Map.csv")

In [6]:
bb_usage.drop(["Unnamed: 0"],axis=1,inplace=True)
user_all_df.drop(["Unnamed: 0"],axis=1,inplace=True)
user_profile.drop(["Unnamed: 0"],axis=1,inplace=True)
user_bb_map.drop(["Unnamed: 0"],axis=1,inplace=True)

In [7]:
bb_usage = bb_usage.merge(user_all_df[["event_source.hash","ACCOUNT_NUM.hash"]], on="event_source.hash", how="left")
bb_usage.drop(["event_source.hash"],axis=1, inplace=True)
bb_usage.drop_duplicates(["ACCOUNT_NUM.hash","rating","year-month"],inplace=True)
bb_usage.dropna(subset=["ACCOUNT_NUM.hash"], inplace=True)

In [8]:
bb_usage = bb_usage.merge(user_bb_map, on="ACCOUNT_NUM.hash", how="left")

In [9]:
bb_usage.dropna(subset=["BB_Package"], inplace=True)

### Exporting Package Names

In [10]:
bb_usage[["rating","BB_Package"]].drop_duplicates(["BB_Package"]).to_csv("data/product_catalog/BB_Names.csv")

In [11]:
rating_df=bb_usage[["ACCOUNT_NUM.hash","BB_Package","rating"]]
rating_df.rename(columns={'rating':'ratings','BB_Package':'package'},inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [12]:
rating_df.reset_index(inplace=True)
rating_df.drop("index", axis=1, inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [13]:
rating_df = rating_df.groupby(["ACCOUNT_NUM.hash","package"]).agg({'ratings':sum})
rating_df.reset_index(inplace=True)

In [14]:
rating_df.head()

,ACCOUNT_NUM.hash,package,ratings
0,000eee57a6c7a02c8aca8b410ea2e287,WEB STARTER,5.8125
1,000f325a72b9d24742237070939b57d1,WEB STARTER,2.6875
2,00103d1ae201c2ddb33b965f44f280b1,FTTH_WEB FAMILY PLUS,8.0000
3,0011c1b02e2403c74c75ae8b5582e018,ANY JOY,1.7500
4,0011c1b02e2403c74c75ae8b5582e018,WEB LITE,1.7500


In [16]:
# storing rating DF
rating_df.to_csv("data/azure/BB_user_ratings.csv")

## User content

In [15]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [16]:
user_profile = user_profile[(user_profile["BB_Package"] != "NO_INFO")]
user_profile.reset_index(drop=True, inplace=True)

In [17]:
user_profile.drop(["BB_Categorical","BB_Scaled","BIRTH_YEAR","Megaline_Month_Subscribed","LTE_Month_Subscribed","VAS_Month_Subscribed"], axis=1, inplace= True)

In [18]:
user_profile.shape

(14836, 114)

## Feature Selection

In [19]:
# data = user_profile.iloc[:,1:]

In [20]:
# mp = []

# for col in data.dtypes:
#     if col == "float64":
#         mp.append(False)
#     else:
#         mp.append(True)
        
# dm = gower.gower_matrix(data, cat_features = mp)

In [ ]:
# pc=PCA(n_components=len(mp)) 
# pc.fit(dm)

In [ ]:
#How mucb variance, captured together
# cumsum_arr = pc.explained_variance_ratio_.cumsum() 

In [ ]:
# pca_df = pd.DataFrame(data.columns.tolist(),columns=["Feature"])
# pca_df["Importance"] = cumsum_arr

In [21]:
# Storing Feature resuced User Profile content
user_profile.to_csv("data/azure/BB_user_content.csv")

## Product (package) content

In [ ]:
product_profile = pd.read_csv("data/product_catalog/Product_Profile_Finalised_CSV.csv")

In [ ]:
product_df=product_profile.iloc[:,[0,2,3,4,6,7,8,9,10,11,12,13,15,36,38,39,40,43,44]]

In [23]:
temp_package_df=rating_df["package"]

In [24]:
bb_products_df = product_df.merge(temp_package_df.drop_duplicates(), left_on=['Product_DB_ID'], right_on=['package'] ,
                   how='left', indicator=True)

In [25]:
bbp_df = bb_products_df[(bb_products_df["_merge"]=="both")]

In [26]:
bbp_df.drop(["package","_merge"],axis=1,inplace=True)
bbp_df.rename(columns={'Product_ID':'package'},inplace=True)
bbp_df.reset_index(drop=True, inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [27]:
cols =["BB_Data_Standard","BB_Data_Free","BB_Data_Anytime","BB_Data_Unlimited"]

In [28]:
bbp_df[cols] = bbp_df[cols].fillna(0)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
scaler = MinMaxScaler()
for col in cols:
    bbp_df[col] = scaler.fit_transform(bbp_df[col].values.reshape(-1,1))

/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_24001/277134799.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbp_df[col] = scaler.fit_transform(bbp_df[col].values.reshape(-1,1))
/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_24001/277134799.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbp_df[col] = scaler.fit_transform(bbp_df[col].values.reshape(-1,1))
/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_24001/277134799.py:3: SettingWithCopyWarning: 
A value is try

In [30]:
# Droping due to all null
bbp_df.dropna(how="all",axis=0,inplace=True)
bbp_df.dropna(how="all",axis=1,inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [31]:
bbp_df.rename(columns={"Product_DB_ID":"package"},inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [32]:
bbp_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   package              59 non-null     object 
 1   Base_Type            59 non-null     object 
 2   Pricing_Type         59 non-null     object 
 3   Package_Type         59 non-null     object 
 4   Title                59 non-null     object 
 5   BB_Data_Standard     59 non-null     float64
 6   BB_Data_Free         59 non-null     float64
 7   BB_Data_Anytime      59 non-null     float64
 8   BB_Data_Unlimited    59 non-null     float64
 9   BB_Connection_Type   59 non-null     object 
 10  BB_Connection_Speed  59 non-null     object 
 11  Price                59 non-null     object 
 12  Subscription_Type    59 non-null     object 
 13  Recidence_Type       59 non-null     object 
 14  Tax_Status           59 non-null     object 
 15  Conditions           59 non-null     objec

In [33]:
bbp_df.to_csv("data/azure/BB_package_content.csv")